# Module 04: Azure ML Pipelines and Deployment

**Difficulty**: ⭐⭐⭐
**Estimated Time**: 60 minutes
**Prerequisites**: 
- [Module 03: Azure ML Studio Introduction](03_azure_ml_studio_introduction.ipynb)
- Understanding of ML pipelines and CI/CD concepts
- Basic knowledge of REST APIs

## Learning Objectives

By the end of this notebook, you will be able to:
1. Create and orchestrate Azure ML Pipelines with components
2. Schedule pipelines with triggers and automate workflows
3. Deploy models to online (real-time) endpoints
4. Use batch endpoints for large-scale inference
5. Integrate MLflow for experiment tracking and model management
6. Implement model registration and versioning strategies

## 1. Setup and Introduction

### Azure ML Pipelines Overview

Pipelines enable building reproducible, automated ML workflows.

```
┌─────────────────────────────────────────────────────────┐
│         Azure ML Pipeline Architecture                   │
├─────────────────────────────────────────────────────────┤
│                                                         │
│  Pipeline = Sequence of Components                      │
│                                                         │
│  ┌─────────────┐    ┌─────────────┐    ┌──────────┐  │
│  │ Component 1 │───▶│ Component 2 │───▶│Component │  │
│  │ Data Prep   │    │  Training   │    │ Evaluate │  │
│  └─────────────┘    └─────────────┘    └──────────┘  │
│                                                         │
│  Each Component:                                        │
│  ├─ Has inputs and outputs                             │
│  ├─ Runs in its own environment                        │
│  ├─ Is reusable across pipelines                       │
│  └─ Can run in parallel                                │
│                                                         │
│  Pipeline Execution:                                    │
│  ├─ Triggered manually or scheduled                    │
│  ├─ Runs on compute clusters                           │
│  ├─ Tracked and versioned                              │
│  └─ Outputs logged to workspace                        │
└─────────────────────────────────────────────────────────┘
```

In [ ]:
# Setup and imports
import numpy as np
import pandas as pd
import json
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
np.random.seed(42)

In [ ]:
# Mock Azure ML components for demonstration
# In production, use: from azure.ai.ml import command, Input, Output

class MockPipelineComponent:
    """Simulates Azure ML pipeline component"""
    
    def __init__(self, name, inputs, outputs, command):
        self.name = name
        self.inputs = inputs
        self.outputs = outputs
        self.command = command
        self.status = 'NotStarted'
    
    def execute(self):
        """Simulate component execution"""
        self.status = 'Running'
        print(f"Executing: {self.name}")
        print(f"Command: {self.command}")
        self.status = 'Completed'
        return {'status': self.status}

## 2. Creating Pipeline Components

Components are reusable building blocks for pipelines.

### Component Structure

```yaml
name: data_preparation
display_name: Data Preparation
version: 1
type: command
inputs:
  raw_data:
    type: uri_folder
outputs:
  clean_data:
    type: uri_folder
code: ./src
environment: azureml:sklearn-env:1
command: >-
  python prep_data.py 
  --input ${{inputs.raw_data}} 
  --output ${{outputs.clean_data}}
```

In [ ]:
# Define pipeline components

def create_data_prep_component():
    """Component for data preparation"""
    component_spec = {
        'name': 'data_preparation',
        'display_name': 'Data Preparation',
        'version': '1',
        'type': 'command',
        'inputs': {
            'raw_data': {'type': 'uri_folder'},
            'test_split_ratio': {'type': 'number', 'default': 0.2}
        },
        'outputs': {
            'train_data': {'type': 'uri_folder'},
            'test_data': {'type': 'uri_folder'}
        },
        'code': './components/prep',
        'environment': 'azureml:sklearn-env:1',
        'command': 'python prep.py --input ${{inputs.raw_data}} --train ${{outputs.train_data}} --test ${{outputs.test_data}} --ratio ${{inputs.test_split_ratio}}'
    }
    return component_spec

def create_training_component():
    """Component for model training"""
    component_spec = {
        'name': 'train_model',
        'display_name': 'Model Training',
        'version': '1',
        'type': 'command',
        'inputs': {
            'training_data': {'type': 'uri_folder'},
            'learning_rate': {'type': 'number', 'default': 0.01},
            'n_estimators': {'type': 'integer', 'default': 100}
        },
        'outputs': {
            'model_output': {'type': 'uri_folder'}
        },
        'code': './components/train',
        'environment': 'azureml:sklearn-env:1',
        'command': 'python train.py --data ${{inputs.training_data}} --lr ${{inputs.learning_rate}} --estimators ${{inputs.n_estimators}} --output ${{outputs.model_output}}'
    }
    return component_spec

prep_comp = create_data_prep_component()
train_comp = create_training_component()

print("Data Preparation Component:")
print(json.dumps(prep_comp, indent=2))

In [ ]:
# Build pipeline from components

def create_training_pipeline():
    """
    Assemble components into complete training pipeline
    Data Prep → Training → Evaluation → Registration
    """
    pipeline_spec = {
        'name': 'classification_training_pipeline',
        'display_name': 'End-to-End Classification Pipeline',
        'description': 'Pipeline for training and evaluating classification model',
        'settings': {
            'default_compute': 'training-cluster',
            'default_datastore': 'workspaceblobstore'
        },
        'inputs': {
            'pipeline_input_data': {
                'type': 'uri_folder',
                'path': 'azureml://datastores/workspaceblobstore/paths/data/'
            },
            'test_split_ratio': {'type': 'number', 'default': 0.2},
            'learning_rate': {'type': 'number', 'default': 0.01}
        },
        'jobs': {
            'prep_step': {
                'component': 'data_preparation:1',
                'inputs': {
                    'raw_data': '${{parent.inputs.pipeline_input_data}}',
                    'test_split_ratio': '${{parent.inputs.test_split_ratio}}'
                }
            },
            'train_step': {
                'component': 'train_model:1',
                'inputs': {
                    'training_data': '${{parent.jobs.prep_step.outputs.train_data}}',
                    'learning_rate': '${{parent.inputs.learning_rate}}'
                }
            },
            'evaluate_step': {
                'component': 'evaluate_model:1',
                'inputs': {
                    'model': '${{parent.jobs.train_step.outputs.model_output}}',
                    'test_data': '${{parent.jobs.prep_step.outputs.test_data}}'
                }
            }
        }
    }
    return pipeline_spec

pipeline = create_training_pipeline()
print("Pipeline Configuration:")
print(json.dumps(pipeline, indent=2))

## 3. Pipeline Scheduling and Triggers

Automate pipeline execution with schedules and event-based triggers.

### Scheduling Options

```
┌─────────────────────────────────────────────────────────┐
│          Pipeline Triggers & Schedules                   │
├─────────────────────────────────────────────────────────┤
│                                                         │
│  1. Time-based Schedule                                 │
│     ┌──────────────────────────────────┐               │
│     │ Cron: 0 2 * * *  (Daily 2am)    │               │
│     │ Frequency: Hourly/Daily/Weekly   │               │
│     └──────────────────────────────────┘               │
│                                                         │
│  2. Data Change Trigger                                 │
│     ┌──────────────────────────────────┐               │
│     │ New file in Blob Storage         │               │
│     │ Dataset version updated          │               │
│     └──────────────────────────────────┘               │
│                                                         │
│  3. Manual Trigger                                      │
│     ┌──────────────────────────────────┐               │
│     │ Azure Portal                     │               │
│     │ SDK/CLI                          │               │
│     │ REST API                         │               │
│     └──────────────────────────────────┘               │
│                                                         │
│  4. CI/CD Integration                                   │
│     ┌──────────────────────────────────┐               │
│     │ GitHub Actions                   │               │
│     │ Azure DevOps Pipelines           │               │
│     │ Jenkins                          │               │
│     └──────────────────────────────────┘               │
└─────────────────────────────────────────────────────────┘
```

In [ ]:
# Create time-based schedule

def create_pipeline_schedule():
    """Configure recurring pipeline schedule"""
    schedule_config = {
        'name': 'daily_retraining_schedule',
        'pipeline_id': 'classification_training_pipeline:1',
        'recurrence': {
            'frequency': 'Day',  # Hour, Day, Week, Month
            'interval': 1,  # Every N days
            'schedule': {
                'hours': [2],  # Run at 2 AM
                'minutes': [0]
            },
            'time_zone': 'UTC'
        },
        'enabled': True,
        'description': 'Retrain model daily with fresh data'
    }
    
    # Alternative: Cron expression
    cron_config = {
        'name': 'weekly_retraining_cron',
        'pipeline_id': 'classification_training_pipeline:1',
        'cron_expression': '0 2 * * 0',  # Every Sunday at 2 AM
        'time_zone': 'UTC',
        'enabled': True
    }
    
    return schedule_config, cron_config

recurrence, cron = create_pipeline_schedule()
print("Recurrence Schedule:")
print(json.dumps(recurrence, indent=2))
print("\nCron Schedule:")
print(json.dumps(cron, indent=2))

In [ ]:
# Simulate scheduled pipeline execution

class PipelineScheduler:
    """Simulates Azure ML pipeline scheduling"""
    
    def __init__(self, pipeline_name, schedule_config):
        self.pipeline_name = pipeline_name
        self.schedule = schedule_config
        self.run_history = []
    
    def calculate_next_run(self, current_time):
        """Calculate next scheduled run time"""
        interval = self.schedule['recurrence']['interval']
        frequency = self.schedule['recurrence']['frequency']
        
        if frequency == 'Day':
            next_run = current_time + timedelta(days=interval)
        elif frequency == 'Hour':
            next_run = current_time + timedelta(hours=interval)
        else:
            next_run = current_time + timedelta(weeks=interval)
        
        return next_run
    
    def trigger_run(self, run_time):
        """Trigger pipeline execution"""
        run_id = f"run_{len(self.run_history) + 1}"
        run_info = {
            'run_id': run_id,
            'pipeline': self.pipeline_name,
            'triggered_at': run_time,
            'trigger_type': 'scheduled',
            'status': 'Running'
        }
        self.run_history.append(run_info)
        print(f"✓ Triggered {run_id} at {run_time}")
        return run_id

In [ ]:
# Test pipeline scheduling
scheduler = PipelineScheduler(
    pipeline_name='classification_training_pipeline',
    schedule_config=recurrence
)

# Simulate scheduled runs
current_time = datetime.now()
print("Simulating scheduled pipeline runs:\n")

for i in range(5):
    next_run_time = scheduler.calculate_next_run(current_time)
    scheduler.trigger_run(next_run_time)
    current_time = next_run_time

print(f"\nTotal runs: {len(scheduler.run_history)}")

## 4. Online Endpoint Deployment

Deploy models for real-time predictions via REST API.

### Online Endpoint Architecture

```
┌─────────────────────────────────────────────────────────┐
│           Azure ML Online Endpoint                       │
├─────────────────────────────────────────────────────────┤
│                                                         │
│  Client App                                             │
│  ┌──────────┐                                           │
│  │ POST     │                                           │
│  │ /score   │                                           │
│  └────┬─────┘                                           │
│       │ HTTPS + Auth                                    │
│       ▼                                                 │
│  ┌─────────────────────────────────────┐               │
│  │  Managed Online Endpoint            │               │
│  │  - Auto-scaling                     │               │
│  │  - Load balancing                   │               │
│  │  - Monitoring                       │               │
│  │  - Managed identity                 │               │
│  └─────────────────────────────────────┘               │
│               │                                         │
│               ▼                                         │
│  ┌──────────────────┐  ┌──────────────────┐           │
│  │ Deployment v1    │  │ Deployment v2    │           │
│  │ (90% traffic)    │  │ (10% traffic)    │           │
│  │                  │  │                  │           │
│  │ Model: v1.0      │  │ Model: v2.0      │           │
│  │ Instances: 2     │  │ Instances: 1     │           │
│  └──────────────────┘  └──────────────────┘           │
└─────────────────────────────────────────────────────────┘
```

In [ ]:
# Create online endpoint configuration

def create_online_endpoint():
    """Configure managed online endpoint"""
    endpoint_config = {
        'name': 'classification-endpoint',
        'description': 'Real-time classification endpoint',
        'auth_mode': 'key',  # or 'aml_token'
        'properties': {
            'enforce_https': True,
            'public_network_access': 'enabled'  # or 'disabled' for private
        },
        'tags': {
            'model': 'classifier',
            'version': '1.0',
            'environment': 'production'
        }
    }
    return endpoint_config

def create_deployment():
    """Configure deployment within endpoint"""
    deployment_config = {
        'name': 'blue-deployment',
        'endpoint_name': 'classification-endpoint',
        'model': 'azureml:classifier-model:1',
        'environment': 'azureml:sklearn-env:1',
        'code_configuration': {
            'code': './scoring',
            'scoring_script': 'score.py'
        },
        'instance_type': 'Standard_DS3_v2',
        'instance_count': 2,
        'app_insights_enabled': True,
        'scale_settings': {
            'type': 'target_utilization',
            'min_instances': 1,
            'max_instances': 5,
            'target_utilization_percentage': 70
        },
        'liveness_probe': {
            'initial_delay': 10,
            'period': 10,
            'timeout': 2,
            'success_threshold': 1,
            'failure_threshold': 3
        }
    }
    return deployment_config

endpoint = create_online_endpoint()
deployment = create_deployment()

print("Online Endpoint:")
print(json.dumps(endpoint, indent=2))
print("\nDeployment Configuration:")
print(json.dumps(deployment, indent=2))

In [ ]:
# Simulate online endpoint invocation

class OnlineEndpoint:
    """Simulates Azure ML online endpoint"""
    
    def __init__(self, endpoint_uri, auth_key):
        self.endpoint_uri = endpoint_uri
        self.auth_key = auth_key
        self.request_count = 0
        self.latencies = []
    
    def invoke(self, data):
        """
        Invoke endpoint for real-time prediction
        
        Args:
            data: Input features for prediction
        
        Returns:
            Prediction result with metadata
        """
        # Simulate API call
        import time
        start_time = time.time()
        
        # Simulate prediction
        prediction = np.random.choice([0, 1], p=[0.6, 0.4])
        probability = np.random.rand()
        
        latency = (time.time() - start_time) * 1000
        self.latencies.append(latency)
        self.request_count += 1
        
        response = {
            'prediction': int(prediction),
            'probability': float(probability),
            'model_version': '1.0',
            'latency_ms': latency
        }
        
        return response
    
    def get_stats(self):
        """Get endpoint statistics"""
        return {
            'total_requests': self.request_count,
            'avg_latency_ms': np.mean(self.latencies) if self.latencies else 0,
            'p95_latency_ms': np.percentile(self.latencies, 95) if self.latencies else 0
        }

In [ ]:
# Test online endpoint
endpoint = OnlineEndpoint(
    endpoint_uri='https://classification-endpoint.eastus.inference.ml.azure.com/score',
    auth_key='[REDACTED]'
)

# Make sample predictions
print("Making predictions via online endpoint:\n")
for i in range(5):
    test_data = np.random.randn(10).tolist()
    result = endpoint.invoke(test_data)
    print(f"Request {i+1}: Prediction={result['prediction']}, "
          f"Probability={result['probability']:.3f}, "
          f"Latency={result['latency_ms']:.2f}ms")

# Display statistics
stats = endpoint.get_stats()
print("\nEndpoint Statistics:")
print(json.dumps(stats, indent=2))

## 5. Batch Endpoints

Process large datasets efficiently with batch inference.

### Batch Endpoint Architecture

```
┌─────────────────────────────────────────────────────────┐
│            Batch Endpoint Workflow                       │
├─────────────────────────────────────────────────────────┤
│                                                         │
│  1. Upload Data                                         │
│     Blob Storage: data/input/batch_001.csv              │
│                                                         │
│  2. Invoke Batch Endpoint                               │
│     ┌─────────────────────────────────┐                │
│     │ Batch Job                       │                │
│     │ - Reads from Blob               │                │
│     │ - Processes in chunks           │                │
│     │ - Parallel processing           │                │
│     │ - Auto-scales compute           │                │
│     └─────────────────────────────────┘                │
│                                                         │
│  3. Write Results                                       │
│     Blob Storage: data/output/predictions.csv           │
│                                                         │
│  Benefits:                                              │
│  - Process millions of records                          │
│  - Cost-effective (compute released after job)          │
│  - Automatic retry on failure                           │
│  - Parallel processing                                  │
└─────────────────────────────────────────────────────────┘
```

In [ ]:
# Create batch endpoint configuration

def create_batch_endpoint():
    """Configure batch endpoint for large-scale inference"""
    endpoint_config = {
        'name': 'batch-scoring-endpoint',
        'description': 'Batch inference for large datasets',
        'defaults': {
            'deployment_name': 'production-batch'
        }
    }
    return endpoint_config

def create_batch_deployment():
    """Configure batch deployment"""
    deployment_config = {
        'name': 'production-batch',
        'endpoint_name': 'batch-scoring-endpoint',
        'model': 'azureml:classifier-model:1',
        'environment': 'azureml:sklearn-env:1',
        'code_configuration': {
            'code': './batch-scoring',
            'scoring_script': 'batch_score.py'
        },
        'compute': 'training-cluster',
        'resources': {
            'instance_count': 4  # Parallel processing
        },
        'max_concurrency_per_instance': 4,
        'mini_batch_size': 10,  # Process 10 files per batch
        'output_action': 'append_row',
        'output_file_name': 'predictions.csv',
        'retry_settings': {
            'max_retries': 3,
            'timeout': 300
        },
        'error_threshold': 10,  # Fail job if >10 files fail
        'logging_level': 'info'
    }
    return deployment_config

batch_endpoint = create_batch_endpoint()
batch_deployment = create_batch_deployment()

print("Batch Endpoint:")
print(json.dumps(batch_endpoint, indent=2))
print("\nBatch Deployment:")
print(json.dumps(batch_deployment, indent=2))

In [ ]:
# Simulate batch inference job

class BatchInferenceJob:
    """Simulates Azure ML batch inference"""
    
    def __init__(self, num_files, instances=4):
        self.num_files = num_files
        self.instances = instances
        self.processed = 0
        self.failed = 0
    
    def run(self):
        """Execute batch inference job"""
        print(f"Starting batch job: {self.num_files} files")
        print(f"Compute instances: {self.instances}\n")
        
        files_per_instance = self.num_files // self.instances
        
        for i in range(self.instances):
            start_idx = i * files_per_instance
            end_idx = start_idx + files_per_instance
            print(f"Instance {i+1}: Processing files {start_idx}-{end_idx}")
            
            # Simulate some failures
            failures = np.random.randint(0, 2)
            self.processed += files_per_instance - failures
            self.failed += failures
        
        success_rate = (self.processed / self.num_files) * 100
        print(f"\n✓ Job completed")
        print(f"Processed: {self.processed}/{self.num_files} ({success_rate:.1f}%)")
        print(f"Failed: {self.failed}")
        
        return {
            'status': 'Completed',
            'processed': self.processed,
            'failed': self.failed
        }

In [ ]:
# Run batch inference simulation
batch_job = BatchInferenceJob(num_files=1000, instances=4)
result = batch_job.run()

print("\n💰 Cost Comparison:")
print("Batch endpoint: Pay only during job execution")
print("Online endpoint: Continuous billing even when idle")
print("\nUse batch for: Large datasets, scheduled scoring, no real-time needs")

## 6. MLflow Integration

MLflow provides experiment tracking, model management, and deployment.

### MLflow in Azure ML

```
┌─────────────────────────────────────────────────────────┐
│              MLflow Integration                          │
├─────────────────────────────────────────────────────────┤
│                                                         │
│  Training Script                                        │
│  ┌─────────────────────────────────────┐               │
│  │ import mlflow                       │               │
│  │                                     │               │
│  │ mlflow.autolog()                    │               │
│  │                                     │               │
│  │ with mlflow.start_run():            │               │
│  │   mlflow.log_param("lr", 0.01)      │               │
│  │   mlflow.log_metric("auc", 0.95)    │               │
│  │   mlflow.sklearn.log_model(model)   │               │
│  └─────────────────────────────────────┘               │
│               │                                         │
│               ▼                                         │
│  Azure ML Workspace                                     │
│  ┌─────────────────────────────────────┐               │
│  │ Experiments                         │               │
│  │ - Parameters logged                 │               │
│  │ - Metrics tracked                   │               │
│  │ - Artifacts stored                  │               │
│  │ - Models registered                 │               │
│  └─────────────────────────────────────┘               │
└─────────────────────────────────────────────────────────┘
```

In [ ]:
# Simulate MLflow tracking

class MLflowTracker:
    """Simulates MLflow experiment tracking"""
    
    def __init__(self, experiment_name):
        self.experiment_name = experiment_name
        self.runs = []
        self.current_run = None
    
    def start_run(self, run_name=None):
        """Start new MLflow run"""
        run_id = f"run_{len(self.runs) + 1}"
        self.current_run = {
            'run_id': run_id,
            'run_name': run_name or run_id,
            'experiment': self.experiment_name,
            'params': {},
            'metrics': {},
            'artifacts': [],
            'start_time': datetime.now()
        }
        print(f"Started MLflow run: {run_id}")
    
    def log_param(self, key, value):
        """Log parameter"""
        if self.current_run:
            self.current_run['params'][key] = value
            print(f"  Param: {key}={value}")
    
    def log_metric(self, key, value, step=None):
        """Log metric"""
        if self.current_run:
            if key not in self.current_run['metrics']:
                self.current_run['metrics'][key] = []
            self.current_run['metrics'][key].append({
                'value': value,
                'step': step
            })
            print(f"  Metric: {key}={value}")
    
    def log_artifact(self, artifact_path):
        """Log artifact (model, plot, etc.)"""
        if self.current_run:
            self.current_run['artifacts'].append(artifact_path)
            print(f"  Artifact: {artifact_path}")
    
    def end_run(self):
        """End current run"""
        if self.current_run:
            self.current_run['end_time'] = datetime.now()
            self.runs.append(self.current_run)
            print(f"Ended run: {self.current_run['run_id']}\n")
            self.current_run = None

In [ ]:
# Simulate training with MLflow tracking
mlflow = MLflowTracker('classification-experiment')

# Run experiment with different parameters
for lr in [0.001, 0.01, 0.1]:
    mlflow.start_run(run_name=f'lr_{lr}')
    
    # Log parameters
    mlflow.log_param('learning_rate', lr)
    mlflow.log_param('n_estimators', 100)
    mlflow.log_param('max_depth', 10)
    
    # Simulate training and log metrics
    for epoch in range(3):
        # Simulated metrics
        train_loss = 1.0 / (epoch + 1) * (1 / lr)
        val_auc = 0.7 + (epoch * 0.05) + (lr * 0.1)
        
        mlflow.log_metric('train_loss', train_loss, step=epoch)
        mlflow.log_metric('val_auc', val_auc, step=epoch)
    
    # Log model artifact
    mlflow.log_artifact('models/classifier.pkl')
    
    mlflow.end_run()

print(f"Total runs logged: {len(mlflow.runs)}")

## 7. Model Registration and Versioning

Register models for deployment and track versions.

### Model Registry Workflow

```
Training → Evaluation → Registration → Staging → Production
```

In [ ]:
# Model registration

def register_model(model_info):
    """Register model in Azure ML workspace"""
    registration = {
        'name': model_info['name'],
        'version': model_info.get('version', 'auto'),  # Auto-increment
        'path': model_info['path'],
        'type': 'mlflow_model',  # or 'custom_model'
        'description': model_info.get('description', ''),
        'tags': {
            'framework': model_info.get('framework', 'sklearn'),
            'task': model_info.get('task', 'classification'),
            'algorithm': model_info.get('algorithm', ''),
            'training_date': datetime.now().isoformat()
        },
        'properties': {
            'auc': model_info.get('auc', 0),
            'accuracy': model_info.get('accuracy', 0),
            'dataset_version': model_info.get('dataset_version', 1)
        },
        'stage': 'Staging'  # or 'Production', 'Archived'
    }
    return registration

# Register a model
model_registration = register_model({
    'name': 'churn-classifier',
    'path': 'runs/run_1/model',
    'description': 'Customer churn prediction model',
    'framework': 'sklearn',
    'algorithm': 'RandomForest',
    'auc': 0.94,
    'accuracy': 0.89,
    'dataset_version': 2
})

print("Model Registration:")
print(json.dumps(model_registration, indent=2))

In [ ]:
# Simulate model registry

class ModelRegistry:
    """Simulates Azure ML model registry"""
    
    def __init__(self):
        self.models = {}
    
    def register(self, model_name, model_info):
        """Register new model or version"""
        if model_name not in self.models:
            self.models[model_name] = []
        
        version = len(self.models[model_name]) + 1
        model_info['version'] = version
        model_info['registered_at'] = datetime.now()
        
        self.models[model_name].append(model_info)
        print(f"✓ Registered {model_name} version {version}")
        return version
    
    def get_latest(self, model_name, stage=None):
        """Get latest version of model"""
        if model_name not in self.models:
            return None
        
        versions = self.models[model_name]
        if stage:
            versions = [v for v in versions if v.get('stage') == stage]
        
        return versions[-1] if versions else None
    
    def promote_to_production(self, model_name, version):
        """Promote model version to production"""
        if model_name in self.models:
            for model in self.models[model_name]:
                if model['version'] == version:
                    model['stage'] = 'Production'
                    print(f"✓ Promoted {model_name} v{version} to Production")
                    return True
        return False

In [ ]:
# Test model registry
registry = ModelRegistry()

# Register multiple versions
v1 = registry.register('churn-classifier', {
    'auc': 0.89,
    'stage': 'Staging'
})

v2 = registry.register('churn-classifier', {
    'auc': 0.94,
    'stage': 'Staging'
})

v3 = registry.register('churn-classifier', {
    'auc': 0.96,
    'stage': 'Staging'
})

# Promote best version to production
registry.promote_to_production('churn-classifier', v3)

# Get production model
prod_model = registry.get_latest('churn-classifier', stage='Production')
print(f"\nProduction model: v{prod_model['version']}, AUC={prod_model['auc']}")

## Exercises

### Exercise 1: Pipeline Design

Design a complete ML pipeline for fraud detection:
1. Data validation (check schema, quality)
2. Feature engineering (aggregate transactions)
3. Model training (with hyperparameter tuning)
4. Model evaluation (multiple metrics)
5. Conditional registration (only if AUC > 0.90)

Define all components and their connections.

In [ ]:
# Your solution here
def design_fraud_detection_pipeline():
    """
    Design end-to-end fraud detection pipeline
    
    Include:
    - Component definitions
    - Input/output connections
    - Conditional logic
    - Parallel execution where possible
    
    Returns:
        dict with complete pipeline specification
    """
    # TODO: Design pipeline
    pass

# Test your pipeline
# pipeline = design_fraud_detection_pipeline()
# print(json.dumps(pipeline, indent=2))

### Exercise 2: Deployment Strategy

You need to deploy a model with these requirements:
- 10,000 predictions/day (mostly during business hours)
- Must support A/B testing new versions
- Latency < 200ms at p95
- Budget: $500/month

Choose between online endpoint, batch endpoint, or hybrid. Justify your choice.

In [ ]:
# Your solution here
def design_deployment_strategy(requirements):
    """
    Design deployment strategy based on requirements
    
    Consider:
    - Traffic patterns
    - Latency requirements
    - Cost constraints
    - A/B testing needs
    
    Returns:
        dict with deployment strategy and cost estimate
    """
    # TODO: Design strategy
    pass

# Test your strategy
# requirements = {
#     'daily_predictions': 10000,
#     'latency_p95_ms': 200,
#     'budget_monthly': 500,
#     'ab_testing': True
# }
# strategy = design_deployment_strategy(requirements)
# print(json.dumps(strategy, indent=2))

### Exercise 3: Model Governance

Design a model versioning and promotion strategy:
- Automatic registration from pipeline
- Staging environment for testing
- Approval workflow for production
- Rollback capability
- Compliance tracking

Define the complete workflow.

In [ ]:
# Your solution here
def design_model_governance():
    """
    Design model governance workflow
    
    Include:
    - Registration criteria
    - Promotion gates (metrics, approvals)
    - Rollback procedure
    - Audit trail
    
    Returns:
        dict with governance workflow
    """
    # TODO: Design governance
    pass

# Test your workflow
# governance = design_model_governance()
# print(json.dumps(governance, indent=2))

## Summary

In this notebook, you learned:

1. **Pipeline Components**: Building reusable, modular ML workflows
2. **Pipeline Scheduling**: Automating workflows with triggers
3. **Online Endpoints**: Real-time model serving with auto-scaling
4. **Batch Endpoints**: Cost-effective large-scale inference
5. **MLflow Integration**: Experiment tracking and model management
6. **Model Registry**: Versioning and lifecycle management

### Key Takeaways

- Use pipelines for reproducible, automated workflows
- Schedule pipelines for regular retraining
- Choose online vs batch based on latency and cost requirements
- Track experiments with MLflow for better visibility
- Version models and use staging for safe deployments
- Implement approval workflows for production models

### What's Next?

- [Module 05: Google Cloud AI Platform Basics](05_google_cloud_ai_platform_basics.ipynb)
- Explore Azure DevOps integration for CI/CD
- Practice building custom pipeline components
- Implement monitoring and alerting for endpoints

### Additional Resources

- [Azure ML Pipelines](https://docs.microsoft.com/azure/machine-learning/concept-ml-pipelines)
- [Online Endpoints](https://docs.microsoft.com/azure/machine-learning/concept-endpoints)
- [Batch Endpoints](https://docs.microsoft.com/azure/machine-learning/batch-inference/)
- [MLflow on Azure ML](https://docs.microsoft.com/azure/machine-learning/how-to-use-mlflow)